In [1]:
!pip install psycopg2-binary pandas sqlalchemy 

import psycopg2
import sqlalchemy
import pandas
import csv


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# CONEXIÓN
conexion = psycopg2.connect(
        host="postgresql-bancolombia.alwaysdata.net",
        user="bancolombia",
        password="pruebatecnica",
        dbname="bancolombia_prueba")

cursor = conexion.cursor()

# FUNCIÓN PARA EJECUTAR
def execute_command(command):
    cursor.execute(command)
    return None




In [3]:
# CREAR TABLAS
comando_crear_catalogo_activos = """
CREATE TABLE catalogo_activos (
    cod_activo VARCHAR(50) PRIMARY KEY,
    activo VARCHAR(100) NOT NULL
);
"""

comando_crear_catalogo_banca = """
CREATE TABLE catalogo_banca (
    cod_banca VARCHAR(50) PRIMARY KEY,
    banca VARCHAR(100) NOT NULL
);
"""

comando_crear_cat_perfil_riesgo = """
CREATE TABLE cat_perfil_riesgo (
    cod_perfil_riesgo VARCHAR(50) PRIMARY KEY,
    perfil_riesgo VARCHAR(100) NOT NULL
);
"""

comando_crear_historico_aba_macroactivos = """
CREATE TABLE historico_aba_macroactivos (
    id_historico SERIAL PRIMARY KEY,
    
    ingestion_year VARCHAR(20),
    ingestion_month VARCHAR(20),
    ingestion_day VARCHAR(20),
    
    id_sistema_cliente VARCHAR(150),
    macroactivo VARCHAR(50),
    cod_activo VARCHAR(50),
    aba VARCHAR(100), 
    cod_perfil_riesgo VARCHAR(50),
    cod_banca VARCHAR(50),
    year VARCHAR(20),
    month VARCHAR(20)
);
"""

comando_crear_historico_aba_usd_internacional = """
CREATE TABLE historico_aba_usd_internacional (
    id_historico SERIAL PRIMARY KEY,

    ingestion_year VARCHAR(20),
    ingestion_month VARCHAR(20),
    ingestion_day VARCHAR(20),

    id_sistema_cliente VARCHAR(150),
    simbol VARCHAR(50),
    cusip VARCHAR(50),
    isin TEXT,
    nombre_activo TEXT, 
    
    cantidad VARCHAR(100),
    valor_mercado VARCHAR(100),
    
    fecha_vencimiento VARCHAR(200),
    tasa_cupon VARCHAR(20)
);
"""


commands = [comando_crear_catalogo_activos,comando_crear_catalogo_banca,
            comando_crear_cat_perfil_riesgo, comando_crear_historico_aba_macroactivos,
            comando_crear_historico_aba_usd_internacional]

tablas = ['catalogo_activos', 'catalogo_banca',
          'cat_perfil_riesgo', 'historico_aba_macroactivos',
          'historico_aba_usd_internacional']


for tabla in tablas:
    execute_command(f"DROP TABLE IF EXISTS {tabla} CASCADE;")

for command in commands:
    execute_command(command)
    
conexion.commit()


In [4]:
# POBLAR TABLAS (ARCHIVOS EN EL CODESPACE)

# Función para reutilizar código
def poblar_tabla (ruta_archivo_csv,nombre_tabla, columnas):
        archivo_csv = open(ruta_archivo_csv, 'r')

        next(archivo_csv)

        cursor.copy_from(
            archivo_csv,
            nombre_tabla,
            sep=',',
            columns= columnas
        )

        archivo_csv.close()


# VACIAR TABLAS
execute_command(f"TRUNCATE TABLE historico_aba_macroactivos RESTART IDENTITY CASCADE;")
execute_command(f"TRUNCATE TABLE historico_aba_usd_internacional RESTART IDENTITY CASCADE;")

execute_command(f"TRUNCATE TABLE catalogo_activos RESTART IDENTITY CASCADE;")
execute_command(f"TRUNCATE TABLE catalogo_banca RESTART IDENTITY CASCADE;")
execute_command(f"TRUNCATE TABLE cat_perfil_riesgo RESTART IDENTITY CASCADE;")

# POBLAR TABLAS CATÁLOGO 

# Poblar catalogo_activos
ruta_activos = 'Data/catalogo_activos.csv'
tabla_activos = 'catalogo_activos'
columnas_activos = ('activo', 'cod_activo') 
poblar_tabla(ruta_activos, tabla_activos, columnas_activos)

# Poblar catalogo_banca
ruta_banca = 'Data/catalogo_banca.csv'
tabla_banca = 'catalogo_banca'
columnas_banca = ('cod_banca', 'banca')
poblar_tabla(ruta_banca, tabla_banca, columnas_banca)

# Poblar cat_perfil_riesgo
ruta_riesgo = 'Data/cat_perfil_riesgo.csv'
tabla_riesgo = 'cat_perfil_riesgo'
columnas_riesgo = ('cod_perfil_riesgo', 'perfil_riesgo')
poblar_tabla(ruta_riesgo, tabla_riesgo, columnas_riesgo)

# POBLAR TABLAS DE HISTÓRICOS 

# Poblar historico_aba_macroactivos
ruta_hist_macro = 'Data/historico_aba_macroactivos.csv'
tabla_hist_macro = 'historico_aba_macroactivos'
columnas_hist_macro = ('ingestion_year', 'ingestion_month', 'ingestion_day', 
                       'id_sistema_cliente', 'macroactivo', 'cod_activo', 'aba', 
                       'cod_perfil_riesgo', 'cod_banca', 'year', 'month')
poblar_tabla(ruta_hist_macro, tabla_hist_macro, columnas_hist_macro)

# Poblar historico_aba_usd_internacional
ruta_hist_usd_international = 'Data/historico_aba_usd_internacional.csv'
tabla_hist_usd_international = 'historico_aba_usd_internacional'
columnas_hist_usd_international = ('ingestion_year', 'ingestion_month', 'ingestion_day', 
                       'id_sistema_cliente', 'simbol', 'cusip', 'isin', 'nombre_activo' , 
                       'cantidad', 'valor_mercado', 'fecha_vencimiento', 
                       'tasa_cupon')
poblar_tabla(ruta_hist_usd_international, tabla_hist_usd_international, columnas_hist_usd_international)

conexion.commit()

cursor.close()
conexion.close()
